In [1]:
import networkx as nx
from custom import load_data as cf
from networkx.algorithms import bipartite
from nxviz import CircosPlot
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Introduction

Bipartite graphs are graphs that have two (bi-) partitions (-partite) of nodes. Nodes within each partition are not allowed to be connected to one another; rather, they can only be connected to nodes in the other partition.

Bipartite graphs can be useful for modelling relations between two sets of entities. We will explore the construction and analysis of bipartite graphs here.

![bipartite graph](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Simple-bipartite-graph.svg/600px-Simple-bipartite-graph.svg.png)

Let's load a [crime data](http://konect.uni-koblenz.de/networks/moreno_crime) bipartite graph and quickly explore it.

> This bipartite network contains persons who appeared in at least one crime case as either a suspect, a victim, a witness or both a suspect and victim at the same time. A left node represents a person and a right node represents a crime. An edge between two nodes shows that the left node was involved in the crime represented by the right node.

In [ ]:
G = cf.load_crime_network()
list(G.edges(data=True))[0:5]

In [ ]:
list(G.nodes(data=True))[0:10]

## Projections

Bipartite graphs can be projected down to one of the projections. For example, we can generate a person-person graph from the person-crime graph, by declaring that two nodes that share a crime node are in fact joined by an edge.

![bipartite graph](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Simple-bipartite-graph.svg/600px-Simple-bipartite-graph.svg.png)

## Exercise

Find the bipartite projection function in the NetworkX `bipartite` module [docs](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.bipartite.html), and use it to obtain the `unipartite` projection of the bipartite graph. (5 min.)

In [ ]:
person_nodes = 
pG = 
list(pG.nodes(data=True))[0:5]

## Exercise

Try visualizing the person-person crime network by using a Circos plot. Ensure that the nodes are grouped by gender and then by number of connections. (5 min.)

Again, recapping the Circos Plot API:

```python
c = CircosPlot(graph_object, node_color='metadata_key1', node_grouping='metadata_key2', node_order='metadat_key3')
c.draw()
plt.show()  # or plt.savefig('...')
```

In [ ]:
for n, d in pG.nodes(data=True):
    ____________________
c = CircosPlot(______, node_color=_________, node_grouping=_________, node_order=__________)
_________
plt.savefig('images/crime-person.png', dpi=300)

## Exercise

Use a similar logic to extract crime links. (2 min.)

In [ ]:
crime_nodes = _________
cG = _____________  # cG stands for "crime graph"

## Exercise

Can you plot how the crimes are connected, using a Circos plot? Try ordering it by number of connections. (5 min.)

In [ ]:
for n in cG.nodes():
    ___________

c = CircosPlot(___________)
___________
plt.savefig('images/crime-crime.png', dpi=300)

## Exercise

NetworkX also implements centrality measures for bipartite graphs, which allows you to obtain their metrics without first converting to a particular projection. This is useful for exploratory data analysis. 

Try the following challenges, referring to the [API documentation](https://networkx.github.io/documentation/networkx-1.9/reference/algorithms.bipartite.html) to help you:

1. Which crimes have the most number of people involved?
1. Which people are involved in the most number of crimes?

Exercise total: 5 min.

In [ ]:
# Degree Centrality
bpdc = _______________________
sorted(___________, key=lambda x: ___, reverse=True)

# Bonus Lecture: Matrix Representation

Bipartite graphs have a natural matrix representation, known as the **biadjacency matrix**. Nodes on one partition are the rows, and nodes on the other partition are the columns.

NetworkX's `bipartite` module provides a function for computing the biadjacency matrix of a bipartite graph.

Let's start by looking at a toy bipartite graph, a "customer-product" purchase record graph, with 4 products and 3 customers. The matrix representation might be as follows:

In [1]:
import numpy as np
# Rows = customers, columns = products, 1 = customer purchased product, 0 = customer did not purchase product.
cp_mat = np.array([[0, 1, 0, 0],
                   [1, 0, 1, 0],
                   [0, 1, 1, 1]])


From this "bi-adjacency" matrix, one can compute the projection onto the customers, matrix multiplying the matrix with its transpose.

In [ ]:
c_mat = cp_mat @ cp_mat.T  # c_mat means "customer matrix"
c_mat

**Pause here and read carefully!**

What we get is the connectivity matrix of the customers, based on shared purchases. The diagonals are the degree of the customers in the original graph, i.e. the number of purchases they originally made, and the off-diagonals are the connectivity matrix, based on shared products.

To get the products matrix, we make the transposed matrix the left side of the matrix multiplication.

In [ ]:
p_mat = cp_mat.T @ cp_mat  # p_mat means "product matrix"
p_mat

You may now try to convince yourself that the diagonals are the number of times a customer purchased that product, and the off-diagonals are the connectivity matrix of the products.

### Exercise

Compute the biadjacency matrix of the graph G. Make the rows the "person" nodes, and the columns the "crime" nodes. (2 min.)

**Hint:** You may want to look at the [bipartite documentation](https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.bipartite.html) for the `biadjacency_matrix` function.

In [ ]:
mat = bipartite.____________(G, row_order=_________, column_order=_________)
mat

### Exercise

In the code cells below, time the computation of projection onto the **crime nodes** using matrix multiplications (1st cell) and NetworkX's `bipartite.projected_graph` function. 

In [ ]:
%%timeit
_______ @ _________

In [ ]:
%%timeit 
____________.___________(______, _________)

Note how the matrix multiplication is much faster. The tradeoff, though, is that we lose the rich metadata that might be encoded on the nodes and edges of the graph.

### Demo

Remember how the diagonal encodes the original degree of any given node in the graph. Let's use that fact to find the node that was most connected.

In [ ]:
person_projection = (mat @ mat.T)
person_projection.diagonal()
noi = np.argmax(person_projection.diagonal())  # this is the node that has the highest degree.

person_projection[noi, noi]   # the original degree in the bipartite graph.

In [ ]:
len(G.neighbors(sorted(person_nodes)[noi]))  # original number of neighbors

Thus far, this is just a teaser as to what you can do with the matrix representation of a graph! I hope it's whetted your appetite for more!